In [15]:
!pip install imbalanced-learn


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [16]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [17]:
!gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

Copying file:///home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet...
- [1 files][  7.1 GiB/  7.1 GiB]                                                
Operation completed over 1 objects/7.1 GiB.                                      


In [18]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
# base_path = 'C:/Eugenio/Maestria/DMEyF/'
base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet'

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)

In [19]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [20]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105, 202106]

data = data[data['foto_mes'].isin(meses_train)]

In [21]:
data.shape

(3773357, 679)

In [22]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201906,0.990169,0.009831
1,201907,0.990071,0.009929
2,201908,0.990691,0.009309
3,201909,0.991594,0.008406
4,201910,0.991169,0.008831
5,201911,0.990207,0.009793
6,201912,0.990530,0.009470
7,202001,0.992311,0.007689
8,202002,0.995289,0.004711
9,202003,0.996224,0.003776


In [23]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

Original proportion:0.009830786617744957
Original proportion:0.009929316728374285
Original proportion:0.009309232346378822
Original proportion:0.008405676251172625
Original proportion:0.008830716553752506
Original proportion:0.009793245689313246
Original proportion:0.009469575788598118
Original proportion:0.007689315532834142
Original proportion:0.004710792677538424
Original proportion:0.003776212539168162
Original proportion:0.006071847976940322
Original proportion:0.007701919199264847
Original proportion:0.00816873378122622
Original proportion:0.007504943375876326
Original proportion:0.006468947777254263
Original proportion:0.006554230256802066
Original proportion:0.006592333360462277
Original proportion:0.0070672257406278385
Original proportion:0.007942993697609054
Original proportion:0.008764025526767309
Original proportion:0.011079276465452578
Original proportion:0.012224699880868741
Original proportion:0.01323054421354135
Original proportion:0.012756419212382231
Original proporti

In [24]:
data.shape

(410019, 679)

In [25]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

,foto_mes,proportion_0,proportion_1
0,201906,0.910488,0.089512
1,201907,0.909673,0.090327
2,201908,0.914833,0.085167
3,201909,0.922459,0.077541
4,201910,0.918857,0.081143
5,201911,0.910799,0.089201
6,201912,0.913495,0.086505
7,202001,0.928594,0.071406
8,202002,0.955009,0.044991
9,202003,0.963611,0.036389


In [26]:
output_file = 'competencia_02_fe_v01_undersampled_10_24M.parquet'

data.to_parquet('/home/eanegrin/datasets/' + output_file, index=False)

In [27]:
!gsutil cp /home/eanegrin/datasets/competencia_02_fe_v01_undersampled_10_24M.parquet /home/eanegrin/buckets/b1/datasets/

Copying file:///home/eanegrin/datasets/competencia_02_fe_v01_undersampled_10_24M.parquet...
- [1 files][623.1 MiB/623.1 MiB]                                                
Operation completed over 1 objects/623.1 MiB.                                    
